### Deliverable 1: Preprocessing the Data for a Neural Network

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("./Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN','NAME'],1)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
application_count = application_df.APPLICATION_TYPE.value_counts()
application_count

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T14        3
T25        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Determine which values to replace if counts are less than ...?
replace_application = list(application_count[application_count < 500].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
classification_count = application_df.CLASSIFICATION.value_counts()
classification_count

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4500        1
C2570        1
C2561        1
C1236        1
C6100        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [7]:
# Determine which values to replace if counts are less than ..?
replace_class = list(classification_count[classification_count < 1800].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [8]:
# Look at INCOME_AMT value counts for binning
income_count = application_df.INCOME_AMT.value_counts()
income_count

0                24388
25000-99999       3747
100000-499999     3374
1M-5M              955
1-9999             728
10000-24999        543
10M-50M            240
5M-10M             185
50M+               139
Name: INCOME_AMT, dtype: int64

In [9]:
# Determine which values to replace if counts are less than ..?
replace_income = list(income_count[income_count < 500].index)

# Replace in dataframe
for cls in replace_income:
    application_df.INCOME_AMT = application_df.INCOME_AMT.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.INCOME_AMT.value_counts()

0                24388
25000-99999       3747
100000-499999     3374
1M-5M              955
1-9999             728
Other              564
10000-24999        543
Name: INCOME_AMT, dtype: int64

In [10]:
# Look at AFFILIATION value counts for binning
affiliate_count = application_df.AFFILIATION.value_counts()
affiliate_count

Independent         18480
CompanySponsored    15705
Family/Parent          64
National               33
Regional               13
Other                   4
Name: AFFILIATION, dtype: int64

In [11]:
# Determine which values to replace if counts are less than ..?
replace_affiliate = list(affiliate_count[affiliate_count < 100].index)

# Replace in dataframe
for cls in replace_affiliate:
    application_df.AFFILIATION = application_df.AFFILIATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.AFFILIATION.value_counts()

Independent         18480
CompanySponsored    15705
Other                 114
Name: AFFILIATION, dtype: int64

In [12]:
# Generate our categorical variable lists
application_cat = application_df.dtypes[application_df.dtypes == 'object'].index.tolist()
application_df[application_cat].nunique()

APPLICATION_TYPE          9
AFFILIATION               3
CLASSIFICATION            6
USE_CASE                  5
ORGANIZATION              4
INCOME_AMT                7
SPECIAL_CONSIDERATIONS    2
dtype: int64

In [13]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_Other,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [14]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df, left_index=True, right_index=True)
application_df = application_df.drop(application_cat,1)
application_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_Other,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [15]:
# Split our preprocessed data into our features and target arrays
y = application_df['IS_SUCCESSFUL'].values.reshape(-1,1)
X = application_df.drop(['IS_SUCCESSFUL','SPECIAL_CONSIDERATIONS_Y'],1).values

# Split the preprocessed data into a training and testing dataset
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=42, stratify=y)

In [16]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

### Deliverable 2: Compile, Train and Evaluate the Model

In [17]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 125
hidden_nodes_layer2 = 50

nn = Sequential()

# First hidden layer
nn.add(Dense(units=hidden_nodes_layer1,input_dim=number_input_features,activation='relu'))

# Second hidden layer
nn.add(Dense(units=hidden_nodes_layer2,activation='relu'))

# Output layer
nn.add(Dense(units=1,activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 125)               4750      
_________________________________________________________________
dense_1 (Dense)              (None, 50)                6300      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 51        
Total params: 11,101
Trainable params: 11,101
Non-trainable params: 0
_________________________________________________________________


In [18]:
# Compile the model
nn.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [19]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=75)

Epoch 1/75
804/804 [==============================] - 2s 1ms/step - loss: 0.5818 - accuracy: 0.7029
Epoch 2/75
804/804 [==============================] - 1s 2ms/step - loss: 0.5514 - accuracy: 0.7357
Epoch 3/75
804/804 [==============================] - 1s 1ms/step - loss: 0.5477 - accuracy: 0.7391
Epoch 4/75
804/804 [==============================] - 1s 2ms/step - loss: 0.5495 - accuracy: 0.7313: 0s - loss:
Epoch 5/75
804/804 [==============================] - 1s 1ms/step - loss: 0.5469 - accuracy: 0.7344
Epoch 6/75
804/804 [==============================] - 1s 2ms/step - loss: 0.5476 - accuracy: 0.7325
Epoch 7/75
804/804 [==============================] - 1s 2ms/step - loss: 0.5508 - accuracy: 0.7307: 0s - loss:
Epoch 8/75
804/804 [==============================] - 1s 1ms/step - loss: 0.5445 - accuracy: 0.7373
Epoch 9/75
804/804 [==============================] - 2s 2ms/step - loss: 0.5430 - accuracy: 0.7379
Epoch 10/75
804/804 [==============================] - 1s 2ms/step - loss: 0

In [20]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5637 - accuracy: 0.7243
Loss: 0.5637486577033997, Accuracy: 0.7243148684501648


In [21]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs('checkpoints/Optimization3/',exist_ok=True)
checkpoint_path = 'checkpoints/Optimization3/weights.{epoch:02d}.hdf5'

# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=100)

# Compile the model
nn.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=75,callbacks=[cp_callback])

Epoch 1/75
 98/804 [==>...........................] - ETA: 1s - loss: 0.5318 - accuracy: 0.7468
Epoch 00001: saving model to checkpoints/Optimization3/weights.01.hdf5
180/804 [=====>........................] - ETA: 1s - loss: 0.5344 - accuracy: 0.7444
Epoch 00001: saving model to checkpoints/Optimization3/weights.01.hdf5
273/804 [=========>....................] - ETA: 1s - loss: 0.5350 - accuracy: 0.7439
Epoch 00001: saving model to checkpoints/Optimization3/weights.01.hdf5
384/804 [=============>................] - ETA: 0s - loss: 0.5352 - accuracy: 0.7432
Epoch 00001: saving model to checkpoints/Optimization3/weights.01.hdf5
497/804 [=================>............] - ETA: 0s - loss: 0.5353 - accuracy: 0.7428
Epoch 00001: saving model to checkpoints/Optimization3/weights.01.hdf5
597/804 [=====================>........] - ETA: 0s - loss: 0.5354 - accuracy: 0.7424
Epoch 00001: saving model to checkpoints/Optimization3/weights.01.hdf5
682/804 [========================>.....] - ETA: 0s - 

144/804 [====>.........................] - ETA: 1s - loss: 0.5431 - accuracy: 0.7354
Epoch 00007: saving model to checkpoints/Optimization3/weights.07.hdf5
275/804 [=========>....................] - ETA: 0s - loss: 0.5394 - accuracy: 0.7367
Epoch 00007: saving model to checkpoints/Optimization3/weights.07.hdf5
354/804 [============>.................] - ETA: 0s - loss: 0.5386 - accuracy: 0.7377
Epoch 00007: saving model to checkpoints/Optimization3/weights.07.hdf5
457/804 [================>.............] - ETA: 0s - loss: 0.5372 - accuracy: 0.7389
Epoch 00007: saving model to checkpoints/Optimization3/weights.07.hdf5
553/804 [===================>..........] - ETA: 0s - loss: 0.5368 - accuracy: 0.7392
Epoch 00007: saving model to checkpoints/Optimization3/weights.07.hdf5
668/804 [=======================>......] - ETA: 0s - loss: 0.5361 - accuracy: 0.7397
Epoch 00007: saving model to checkpoints/Optimization3/weights.07.hdf5
764/804 [===========================>..] - ETA: 0s - loss: 0.535

140/804 [====>.........................] - ETA: 1s - loss: 0.5487 - accuracy: 0.7404
Epoch 00013: saving model to checkpoints/Optimization3/weights.13.hdf5
220/804 [=======>......................] - ETA: 1s - loss: 0.5422 - accuracy: 0.7433
Epoch 00013: saving model to checkpoints/Optimization3/weights.13.hdf5
327/804 [===========>..................] - ETA: 0s - loss: 0.5400 - accuracy: 0.7435
Epoch 00013: saving model to checkpoints/Optimization3/weights.13.hdf5
449/804 [===============>..............] - ETA: 0s - loss: 0.5392 - accuracy: 0.7427
Epoch 00013: saving model to checkpoints/Optimization3/weights.13.hdf5
532/804 [==================>...........] - ETA: 0s - loss: 0.5390 - accuracy: 0.7420
Epoch 00013: saving model to checkpoints/Optimization3/weights.13.hdf5
622/804 [======================>.......] - ETA: 0s - loss: 0.5386 - accuracy: 0.7418
Epoch 00013: saving model to checkpoints/Optimization3/weights.13.hdf5
721/804 [=========================>....] - ETA: 0s - loss: 0.538

119/804 [===>..........................] - ETA: 1s - loss: 0.5353 - accuracy: 0.7298
Epoch 00019: saving model to checkpoints/Optimization3/weights.19.hdf5
227/804 [=======>......................] - ETA: 1s - loss: 0.5348 - accuracy: 0.7355
Epoch 00019: saving model to checkpoints/Optimization3/weights.19.hdf5
323/804 [===========>..................] - ETA: 1s - loss: 0.5348 - accuracy: 0.7373
Epoch 00019: saving model to checkpoints/Optimization3/weights.19.hdf5
401/804 [=============>................] - ETA: 0s - loss: 0.5344 - accuracy: 0.7385
Epoch 00019: saving model to checkpoints/Optimization3/weights.19.hdf5
496/804 [=================>............] - ETA: 0s - loss: 0.5337 - accuracy: 0.7398
Epoch 00019: saving model to checkpoints/Optimization3/weights.19.hdf5
598/804 [=====================>........] - ETA: 0s - loss: 0.5331 - accuracy: 0.7407
Epoch 00019: saving model to checkpoints/Optimization3/weights.19.hdf5
694/804 [========================>.....] - ETA: 0s - loss: 0.532

180/804 [=====>........................] - ETA: 1s - loss: 0.5324 - accuracy: 0.7399
Epoch 00025: saving model to checkpoints/Optimization3/weights.25.hdf5
288/804 [=========>....................] - ETA: 0s - loss: 0.5312 - accuracy: 0.7417
Epoch 00025: saving model to checkpoints/Optimization3/weights.25.hdf5
396/804 [=============>................] - ETA: 0s - loss: 0.5309 - accuracy: 0.7427
Epoch 00025: saving model to checkpoints/Optimization3/weights.25.hdf5
467/804 [================>.............] - ETA: 0s - loss: 0.5311 - accuracy: 0.7429
Epoch 00025: saving model to checkpoints/Optimization3/weights.25.hdf5
588/804 [====================>.........] - ETA: 0s - loss: 0.5315 - accuracy: 0.7430
Epoch 00025: saving model to checkpoints/Optimization3/weights.25.hdf5
664/804 [=======================>......] - ETA: 0s - loss: 0.5319 - accuracy: 0.7427
Epoch 00025: saving model to checkpoints/Optimization3/weights.25.hdf5
791/804 [============================>.] - ETA: 0s - loss: 0.532

...

296/804 [==========>...................] - ETA: 1s - loss: 0.5315 - accuracy: 0.7432
Epoch 00049: saving model to checkpoints/Optimization3/weights.49.hdf5
386/804 [=============>................] - ETA: 1s - loss: 0.5317 - accuracy: 0.7432
Epoch 00049: saving model to checkpoints/Optimization3/weights.49.hdf5
498/804 [=================>............] - ETA: 0s - loss: 0.5314 - accuracy: 0.7433
Epoch 00049: saving model to checkpoints/Optimization3/weights.49.hdf5
596/804 [=====================>........] - ETA: 0s - loss: 0.5315 - accuracy: 0.7430
Epoch 00049: saving model to checkpoints/Optimization3/weights.49.hdf5
701/804 [=========================>....] - ETA: 0s - loss: 0.5316 - accuracy: 0.7427
Epoch 00049: saving model to checkpoints/Optimization3/weights.49.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5317 - accuracy: 0.7426
Epoch 50/75
  1/804 [..............................] - ETA: 1s - loss: 0.6066 - accuracy: 0.7188
Epoch 00050: saving model to check

382/804 [=============>................] - ETA: 0s - loss: 0.5269 - accuracy: 0.7450
Epoch 00055: saving model to checkpoints/Optimization3/weights.55.hdf5
449/804 [===============>..............] - ETA: 0s - loss: 0.5275 - accuracy: 0.7449
Epoch 00055: saving model to checkpoints/Optimization3/weights.55.hdf5
557/804 [===================>..........] - ETA: 0s - loss: 0.5281 - accuracy: 0.7448
Epoch 00055: saving model to checkpoints/Optimization3/weights.55.hdf5
655/804 [=======================>......] - ETA: 0s - loss: 0.5284 - accuracy: 0.7448
Epoch 00055: saving model to checkpoints/Optimization3/weights.55.hdf5
755/804 [===========================>..] - ETA: 0s - loss: 0.5287 - accuracy: 0.7446
Epoch 00055: saving model to checkpoints/Optimization3/weights.55.hdf5
804/804 [==============================] - 1s 2ms/step - loss: 0.5289 - accuracy: 0.7444
Epoch 56/75
 78/804 [=>............................] - ETA: 0s - loss: 0.5245 - accuracy: 0.7448
Epoch 00056: saving model to check

452/804 [===============>..............] - ETA: 0s - loss: 0.5213 - accuracy: 0.7512
Epoch 00061: saving model to checkpoints/Optimization3/weights.61.hdf5
524/804 [==================>...........] - ETA: 0s - loss: 0.5227 - accuracy: 0.7497
Epoch 00061: saving model to checkpoints/Optimization3/weights.61.hdf5
646/804 [=======================>......] - ETA: 0s - loss: 0.5248 - accuracy: 0.7478
Epoch 00061: saving model to checkpoints/Optimization3/weights.61.hdf5
733/804 [==========================>...] - ETA: 0s - loss: 0.5258 - accuracy: 0.7470
Epoch 00061: saving model to checkpoints/Optimization3/weights.61.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5264 - accuracy: 0.7465
Epoch 62/75
 41/804 [>.............................] - ETA: 0s - loss: 0.5564 - accuracy: 0.7244
Epoch 00062: saving model to checkpoints/Optimization3/weights.62.hdf5
116/804 [===>..........................] - ETA: 0s - loss: 0.5462 - accuracy: 0.7288
Epoch 00062: saving model to check

529/804 [==================>...........] - ETA: 0s - loss: 0.5279 - accuracy: 0.7476
Epoch 00067: saving model to checkpoints/Optimization3/weights.67.hdf5
600/804 [=====================>........] - ETA: 0s - loss: 0.5284 - accuracy: 0.7470
Epoch 00067: saving model to checkpoints/Optimization3/weights.67.hdf5
725/804 [==========================>...] - ETA: 0s - loss: 0.5292 - accuracy: 0.7460
Epoch 00067: saving model to checkpoints/Optimization3/weights.67.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5295 - accuracy: 0.7456
Epoch 68/75
  1/804 [..............................] - ETA: 1s - loss: 0.5766 - accuracy: 0.6250
Epoch 00068: saving model to checkpoints/Optimization3/weights.68.hdf5
124/804 [===>..........................] - ETA: 0s - loss: 0.5378 - accuracy: 0.7296
Epoch 00068: saving model to checkpoints/Optimization3/weights.68.hdf5
209/804 [======>.......................] - ETA: 0s - loss: 0.5367 - accuracy: 0.7327
Epoch 00068: saving model to check

570/804 [====================>.........] - ETA: 0s - loss: 0.5312 - accuracy: 0.7448
Epoch 00073: saving model to checkpoints/Optimization3/weights.73.hdf5
699/804 [=========================>....] - ETA: 0s - loss: 0.5311 - accuracy: 0.7446
Epoch 00073: saving model to checkpoints/Optimization3/weights.73.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5312 - accuracy: 0.7443
Epoch 74/75
  1/804 [..............................] - ETA: 1s - loss: 0.5004 - accuracy: 0.8125
Epoch 00074: saving model to checkpoints/Optimization3/weights.74.hdf5
102/804 [==>...........................] - ETA: 1s - loss: 0.5333 - accuracy: 0.7478
Epoch 00074: saving model to checkpoints/Optimization3/weights.74.hdf5
186/804 [=====>........................] - ETA: 1s - loss: 0.5336 - accuracy: 0.7464
Epoch 00074: saving model to checkpoints/Optimization3/weights.74.hdf5
307/804 [==========>...................] - ETA: 0s - loss: 0.5335 - accuracy: 0.7445
Epoch 00074: saving model to check

In [22]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5702 - accuracy: 0.7247
Loss: 0.5702239274978638, Accuracy: 0.7246647477149963


In [23]:
# Export our model to HDF5 file
nn.save('AlphabetSoupCharity_Optimization3.h5')